In [1]:
import pandas as pd
from sklearn.metrics import RocCurveDisplay
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from sklearn import metrics
from collections import defaultdict
import joblib

import sys

import time
start_time = time.time()


infile='/Users/irffanalahi/Research/Research_update/SoftRD/largerEXP/genepromdelta.7/Randomforest/developing/BL14_genepromSM_0.7.txt_result_dupindex_binnedstats.pkl_test.pkl'


gtcol='label2'

##### carefully input the model folder#####

modelfol='/Users/irffanalahi/Research/Research_update/SoftRD/largerEXP/genepromdelta.7/Randomforest/developing/BL14_genepromSM_0.7.txt_result_dupindex_binnedstats.pkl_train.pkl_model'


indf=pd.read_pickle(infile)

indf.head()

,index,CD4-others,CD8-others,nB-others,NK-others,Mn-others,mNeu-others,m8-others,DC-others,Eo-others,...,maxscore_beforeCpGweight,maxscoredCT,maxscore,deltabasedfragassignment,filename,label,Last,label2,deltaMedian,deltaMean
952025,mNeu.HWI-ST539_128:8:2208:11901:22353#0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.724654,CD4-others,-0.000000,CD4-others,CD4_fivemixgeneproNotpicard_NR_767146_insilmix...,mNeu,HWI-ST539_128:8:2208:11901:22353#0,NOTCD4,0.000000,0.000000e+00
1300617,nB.HWI-ST700660:310:C7T58ACXX:3:2305:4973:52611,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.745026,CD4-others,-0.000000,CD4-others,CD4_fivemixgeneproNotpicard_NR_767146_insilmix...,nB,HWI-ST700660:310:C7T58ACXX:3:2305:4973:52611,NOTCD4,0.000000,0.000000e+00
1314716,Eo.HWI-ST539:250:C7T3PACXX:1:2106:9545:98822,0.711910,-0.073885,-0.222141,-0.200962,-0.222141,-0.214962,0.017295,-0.210654,-0.057013,...,0.711910,CD4-others,0.711910,CD4-others,CD4_fivemixgeneproNotpicard_NR_767146_insilmix...,Eo,HWI-ST539:250:C7T3PACXX:1:2106:9545:98822,NOTCD4,-0.148910,-7.137148e-17
1078823,CD8.HWI-ST699_224:5:2316:6202:27084#19,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.751897,CD4-others,0.000000,CD4-others,CD4_fivemixgeneproNotpicard_NR_767146_insilmix...,CD8,HWI-ST699_224:5:2316:6202:27084#19,NOTCD4,0.000000,0.000000e+00
663738,CD4.HWI-ST699_224:7:2109:12002:60846#12,1.458179,1.285513,-0.448692,0.051359,-0.232231,-0.369718,-0.239231,-0.312282,-0.439000,...,1.458179,CD4-others,1.458179,CD4-others,CD4_fivemixgeneproNotpicard_NR_767146_insilmix...,CD4,HWI-ST699_224:7:2109:12002:60846#12,CD4,-0.275756,-1.506731e-16


In [2]:
features=['CD4-others', 'CD8-others', 'nB-others', 'NK-others','Mn-others', 'mNeu-others', 'm8-others', 'DC-others', 'Eo-others','Tr-others', 'm4-others', 'Mg-others', 'Er-others', 'mB-others','LENhypoCpG', 'LENhyperCpG', 'total_cpg', 'LENhypoCpG_BY_total_cpg','deltaMedian', 'deltaMean']

In [3]:
scorecolumns=['CD4-others', 'CD8-others', 'nB-others', 'NK-others','Mn-others', 'mNeu-others', 'm8-others', 'DC-others', 'Eo-others','Tr-others', 'm4-others', 'Mg-others', 'Er-others', 'mB-others']

In [4]:
celltypeAUC=defaultdict(list)

for score in scorecolumns:
    
    tempct=score.replace('-others','')
    
 
  
    indfsingleCT=indf[(indf[gtcol]==tempct) | (indf[gtcol]=='NOT'+tempct)].copy()
    
    
    
    indfsingleCT.loc[indfsingleCT[gtcol]==tempct,gtcol]=1
    indfsingleCT.loc[indfsingleCT[gtcol]=='NOT'+tempct,gtcol]=0
    

    
    indfsingleCT[gtcol]=indfsingleCT[gtcol].astype(int)
    
    
  
    testY=indfsingleCT[gtcol]
    testX=indfsingleCT[features]
    
    RDF_classifier =joblib.load(modelfol+"/"+tempct+".pkl")
    
    predict_test=RDF_classifier.predict_proba(testX)
    
    
    
    
    fpr, tpr, thresholds=metrics.roc_curve(testY,predict_test[:,1],pos_label=1)


    tempAUC=metrics.auc(fpr, tpr)
    
    
   
    
   


    
    
    celltypeAUC[tempct]=celltypeAUC[tempct]+[tempAUC]

In [5]:
celltypeAUCdf=pd.DataFrame.from_dict(celltypeAUC, orient='index',columns=['AUC'])
celltypeAUCdf.index.name = 'Celltype'
celltypeAUCdf.to_csv(infile+"_RFAUC.txt",sep="\t")

In [6]:
end_time = time.time()

time_elapsed = (end_time - start_time)

print(time_elapsed)

4.375752925872803
